# Feature Selection

In [ ]:
# Code source: Sebastian Curi and Andreas Krause, based on Jaques Grobler (sklearn demos).
# License: BSD 3 clause

%matplotlib inline
%load_ext autoreload
%autoreload 2
import ipywidgets
from ipywidgets import interact, interactive, interact_manual
import IPython
from IPython.display import display, clear_output
from matplotlib import rcParams

import numpy as np
import matplotlib.pyplot as plt
from utilities.util import gradient_descent
from utilities.load_data import linear_separable_data, circular_separable_data
from utilities import plot_helpers 
from utilities.classifiers import Perceptron, SVM, Logistic
from utilities.regularizers import L1Regularizer, L2Regularizer

from sklearn import svm
from sklearn import datasets
import sklearn
from sklearn.datasets import make_regression
from sklearn.linear_model import Lasso, SGDClassifier, Ridge


## Non-Linear Features

In [ ]:
rcParams['figure.figsize'] = (10, 5)   # Change this if figures look ugly. 
rcParams['font.size'] = 16   # Change this if figures look ugly. 

def laplacian_kernel(X, Y, bw):
    rows = X.shape[0]
    cols = Y.shape[0]
    K = np.zeros((rows, cols))
    for col in range(cols):
        dist = bw * np.linalg.norm(X - Y[col, :], ord=1, axis=1)
        K[:, col] = np.exp(-dist)
    return K

# Our dataset and targets
n_samples = 200  # Number of points per class
tol = 1e-1

def svm_features(dataset, features, reg, bw, deg, noise):
    if dataset is 'blobs':
        X, Y = datasets.make_blobs(n_samples=n_samples, centers=2, random_state=3, cluster_std=10*noise)
    elif dataset is 'circles':
        X, Y = datasets.make_circles(n_samples=n_samples, factor=.5, noise=noise, random_state=42)
    elif dataset is 'moons':
        X, Y = datasets.make_moons(n_samples=n_samples, noise=noise, random_state=42)
    elif dataset == 'xor':
        np.random.seed(42)
        step = int(n_samples/4)
        
        X = np.zeros((n_samples, 2))
        Y = np.zeros(n_samples)
        
        X[0*step:1*step, :] = noise * np.random.randn(step, 2)
        Y[0*step:1*step] = 1
        X[1*step:2*step, :] = np.array([1, 1]) + noise * np.random.randn(step, 2)
        Y[1*step:2*step] = 1
        
        X[2*step:3*step, :] = np.array([0, 1]) + noise * np.random.randn(step, 2)
        Y[2*step:3*step] = -1
        X[3*step:4*step, :] = np.array([1, 0]) + noise * np.random.randn(step, 2)
        Y[3*step:4*step] = -1
    
    elif dataset == 'periodic':
        np.random.seed(42)
        step = int(n_samples/4)
        
        X = np.zeros((n_samples, 2))
        Y = np.zeros(n_samples)
        
        X[0*step:1*step, :] = noise * np.random.randn(step, 2)
        Y[0*step:1*step] = 1
        X[1*step:2*step, :] = np.array([0, 2]) + noise * np.random.randn(step, 2)
        Y[1*step:2*step] = 1
        
        X[2*step:3*step, :] = np.array([0, 1]) + noise * np.random.randn(step, 2)
        Y[2*step:3*step] = -1
        X[3*step:4*step, :] = np.array([0, 3]) + noise * np.random.randn(step, 2)
        Y[3*step:4*step] = -1
        
    X = X[Y <= 1, :]
    Y = Y[Y <=1 ]
    Y[Y==0] = -1
        
    # Add the 1 feature.  
    X = np.concatenate((X, np.ones((X.shape[0], 1))), axis=1)
    plot_support = True
    kernel = features
    if kernel == 'poly':
        gamma = 1
        coef0 = 0
    elif kernel == 'sigmoid':
        gamma = np.power(10., bw)
        coef0 = 0
    elif kernel == 'rbf':
        gamma = np.power(10., -bw)
        coef0 = 0
    elif kernel == 'laplacian':
        gamma = np.power(10., -bw)
        coef0 = 0
        kernel = lambda X, Y: laplacian_kernel(X, Y, gamma)
        plot_support = False

    classifier = svm.SVC(kernel=kernel, C=np.power(10., -reg), gamma=gamma, degree=deg, coef0=coef0, tol=tol)
    classifier.fit(X, Y)

    # plot the line, the points, and the nearest vectors to the plane
    plt.figure()
    plt.clf()
    fig = plt.axes()
    opt = {'marker': 'r*', 'label': '+'}
    plot_helpers.plot_data(X[np.where(Y == 1)[0], 0], X[np.where(Y == 1)[0], 1], fig=fig, options=opt)
    opt = {'marker': 'bs', 'label': '-'}
    plot_helpers.plot_data(X[np.where(Y == -1)[0], 0], X[np.where(Y == -1)[0], 1], fig=fig, options=opt)
    
    if plot_support:
        plt.scatter(classifier.support_vectors_[:, 0], classifier.support_vectors_[:, 1], s=80,
                    facecolors='none', edgecolors='k')

    mins = np.min(X, 0)
    maxs = np.max(X, 0)
    x_min = mins[0] - 1
    x_max = maxs[0] + 1
    y_min = mins[1] - 1
    y_max = maxs[1] + 1

    XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]  
    Xtest = np.c_[XX.ravel(), YY.ravel(), np.ones_like(XX.ravel())]
    Z = classifier.decision_function(Xtest)

    # Put the result into a color plot
    Z = Z.reshape(XX.shape)
    plt.contourf(XX, YY, Z > 0, cmap=plt.cm.jet, alpha=0.3)
    plt.contour(XX, YY, Z, colors=['k', 'k', 'k'], linestyles=['--', '-', '--'], levels=[-.99, 0, .99])

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)


interact(svm_features, 
         dataset=['blobs', 'circles', 'moons', 'xor', 'periodic'],
         features=['poly', 'rbf', 'laplacian'], 
         reg=ipywidgets.FloatSlider(value=-3,
                                    min=-3,
                                    max=3,
                                    step=0.5,
                                    readout_format='.1f',
                                    description='Regularization 10^:',
                                    style={'description_width': 'initial'},
                                    continuous_update=False),
         bw=ipywidgets.FloatSlider(value=-1,
                                    min=-3,
                                    max=3,
                                    step=0.1,
                                    readout_format='.1f',
                                    description='Bandwidth 10^:',
                                    style={'description_width': 'initial'},
                                    continuous_update=False),  
         deg=ipywidgets.IntSlider(
                         value=1,
                         min=1,
                         max=10, 
                         step=1,
                         description='Degree of Poly:',
                         style={'description_width': 'initial'}),
         noise=ipywidgets.FloatSlider(value=0.05,
                                    min=0.01,
                                    max=0.3,
                                    step=0.01,
                                    readout_format='.2f',
                                    description='Noise level:',
                                    style={'description_width': 'initial'},
                                    continuous_update=False),  
        );

# Features Selection

Data are generated according to the following generative model: $$x \sim \mathcal{N}(0, 1), \qquad y = 1 + x + 0.3 \sin(10x).$$

In [ ]:
rcParams['figure.figsize'] = (10, 5)   # Change this if figures look ugly. 
rcParams['font.size'] = 16   # Change this if figures look ugly. 

np.random.seed(42)
num_points = 100
noise = 0.
train_fraction = 0.7  # Train/Validation split = 0.7/0.3

X = np.random.randn(num_points)
Y = 1 + X + 0.3 * np.sin(10 * X) + noise * np.random.randn(num_points)
Xtest = np.linspace(-3, 3, 100)
Ytest = 1 + Xtest + 0.3 * np.sin(10 * Xtest)  # Noiseless


# Split Train into train and validation.
idx = np.arange(0, num_points)
np.random.shuffle(idx)
num_train = int(train_fraction * num_points)

Xtrain, Ytrain = X[idx[:num_train]], Y[idx[:num_train]]
Xval, Yval = X[idx[num_train:]], Y[idx[num_train:]]


fig = plt.subplot(111);
plot_opts = {'x_label': '$x$', 'y_label': '$y$', 'title': 'Generated Data', 'y_lim': [np.min(Y)-0.5, np.max(Y)+0.5]}
plot_helpers.plot_data(X, Y, fig=fig, options=plot_opts)


### Define Features:
We define polynomials $x^p$, with $p \in \left\{0, 1, 2, 3 \right\}$; sine functions $\sin(\omega x)$, with $\omega \in \left\{1, 2, 5, 10 \right\}$; and exponential functions $e^{\alpha x}$, with $\alpha \in \left\{1, 2, 5, 10 \right\}$.

In [ ]:
# Features 
poly = [] 
sin = []
exp = []
for i in [0, 1, 2, 3]:
    poly.append(lambda x, n=i: x**n)
for i in [1, 2, 5, 10]: 
    sin.append(lambda x, n=i: np.sin(n * x))
    exp.append(lambda x, n=i: np.exp(n * x))
               
def build_features(x, feature_list):
    func_list = parse_list(feature_list)
    return np.atleast_2d(np.stack( tuple(feature(x) for feature in func_list) )).T

def get_coefficient(x_train, y_train, feature_list):
    phi_train = build_features(x_train, feature_list)    
    dim = phi_train.shape[-1]
    w_hat = np.dot(np.linalg.pinv(np.dot(phi_train.T, phi_train) + 1e-16 * np.eye(dim)), 
                   np.dot(phi_train.T, y_train))
    return w_hat

def evaluate_features(x_test, y_test, feature_list, w_hat):
    phi_test = build_features(x_test, feature_list)
    error = np.power(y_test - phi_test @ w_hat, 2).mean()
    return error 

def cross_validation(x, y, feature_list, num_folds=5):
    num_points = len(x)
    
    idx = np.arange(num_points)
    folds = np.split(idx, num_folds)  # Careful, this must have equal division.
    error = 0 
    for i_fold in range(num_folds):
        train_idx = np.concatenate(tuple(folds[i] for i in range(num_folds) if i != i_fold))
        val_idx = folds[i_fold]
        
        x_train, y_train = x[train_idx], y[train_idx]
        x_val, y_val = x[val_idx], y[val_idx]
        
        w_hat = get_coefficient(x_train, y_train, feature_list)
        error += evaluate_features(x_val, y_val, feature_list, w_hat)
    
    return error / num_folds


def parse_list(feature_list):
    
    ans = []
    if 'Constant' in feature_list:
        ans.append(poly[0])
    if 'Linear' in feature_list: 
        ans.append(poly[1])
    if 'Squared' in feature_list: 
        ans.append(poly[2])
    if 'Cubic' in feature_list: 
        ans.append(poly[3])
    
    if 'Sin(x)' in feature_list:
        ans.append(sin[0])
    if 'Sin(2x)' in feature_list: 
        ans.append(sin[1])
    if 'Sin(5x)' in feature_list: 
        ans.append(sin[2])
    if 'Sin(10x)' in feature_list: 
        ans.append(sin[3])
    
    if 'Exp(x)' in feature_list:
        ans.append(exp[0])
    if 'Exp(2x)' in feature_list: 
        ans.append(exp[1])
    if 'Exp(5x)' in feature_list: 
        ans.append(exp[2])
    if 'Exp(10x)' in feature_list: 
        ans.append(exp[3])
        
    return ans 

features=['Constant', 'Linear', 'Squared', 'Cubic',
         'Sin(x)', 'Sin(2x)', 'Sin(5x)', 'Sin(10x)',
         'Exp(x)', 'Exp(2x)', 'Exp(5x)', 'Exp(10x)']

## Selection by Hand

In [ ]:
rcParams['figure.figsize'] = (10, 5)   # Change this if figures look ugly. 
rcParams['font.size'] = 16   # Change this if figures look ugly. 
feature_widget = ipywidgets.SelectMultiple(options=features,
    value=['Constant'],
    rows=len(features),
    description='Features',
    disabled=False
)
def feature_selection(feature_list):
    print(feature_list)
    w_hat = get_coefficient(Xtrain, Ytrain, feature_list)
    
    
    fig = plt.subplot(111);
    plot_opts = {'x_label': '$x$', 'y_label': '$y$', 'title': 'Generated Data', 'y_lim': [np.min(Y)-0.5, np.max(Y)+0.5]}
    plot_helpers.plot_data(X, Y, fig=fig, options=plot_opts)
    
    Phival = build_features(Xval, feature_list)
    Phitest = build_features(Xtest, feature_list)
    plt.plot(Xtest, Phitest @ w_hat, 'r-')
    
    print('Coefficients:', w_hat)
    print('Validation Error:', evaluate_features(Xval, Yval, feature_list, w_hat))
    print('Test MSE Error:', evaluate_features(Xtest, Ytest, feature_list, w_hat))
    
        
interact(feature_selection, feature_list=feature_widget);

## Forward Greedy Selection

In [ ]:
rcParams['figure.figsize'] = (10, 5)   # Change this if figures look ugly. 
rcParams['font.size'] = 16   # Change this if figures look ugly. 
np.random.seed(0)

# Generate data
num_points = 100
noise = 0.
num_folds = 5

X = np.random.randn(num_points)
Y = 1 + X + 0.3 * np.sin(10 * X) + noise * np.random.randn(num_points)
Xtest = np.linspace(-3, 3, 100)
Ytest = 1 + Xtest + 0.3 * np.sin(10 * Xtest)  # Noiseless

# Define features
features = {'Constant', 'Linear', 'Squared', 'Cubic', 
            'Sin(x)', 'Sin(2x)', 'Sin(5x)', 'Sin(10x)',
            'Exp(x)', 'Exp(2x)', 'Exp(5x)', 'Exp(10x)'}

feature_set = set()
num_features = 0 
valid_error = float('+Inf')

def next_feature(b):
    global features, feature_set, num_features, valid_error
    best_feature = None
    best_feature_error = float('+Inf')
    
    # Find Best Feature
    for feature in features:
        feature_set.add(feature)
        w_hat = get_coefficient(X, Y, feature_set)
        current_error = cross_validation(X, Y, feature_set, num_folds)
        
        if current_error < best_feature_error:
            best_feature = feature 
            best_feature_error = current_error
        
        feature_set.remove(feature) # Remove feature 
    
    # If validation error increases
    if best_feature_error > valid_error or len(features) == 0:
        print('Finished! Feature set:', feature_set)
        print('Validation error:', valid_error) 
        w_hat = get_coefficient(X, Y, feature_set)
        print('Coefficients', w_hat)
    else:
        # Add feature to feature set. 
        num_features += 1 
        feature_set.add(best_feature)
        valid_error = best_feature_error
        features.remove(best_feature)

        Phitest = build_features(Xtest, feature_set)        
        w_hat = get_coefficient(X, Y, feature_set)
        
        plt.clf()
        clear_output()
        display(button)
        plot_opts = {'x_label': '$x$', 'y_label': '$y$', 'title': 'Num Features {}'.format(num_features), 'y_lim': [np.min(Y)-0.5, np.max(Y)+0.5]}
        plot_helpers.plot_data(X, Y, options=plot_opts)
        plt.plot(Xtest, Phitest @ w_hat, 'r-')
        print('Added Feature', best_feature)
        print('Feature set:', list(feature_set))
        print('Coefficients', w_hat)
        

button = ipywidgets.Button(description="Next Feature")
button.on_click(next_feature)
next_feature(None)

## Backward Greedy Selection

In [ ]:
rcParams['figure.figsize'] = (10, 5)   # Change this if figures look ugly. 
rcParams['font.size'] = 16   # Change this if figures look ugly. 

np.random.seed(0)

# Generate data
num_points = 100
noise = 0
num_folds = 5

X = np.random.randn(num_points)
Y = 1 + X + 0.3 * np.sin(10 * X) + noise * np.random.randn(num_points)
Xtest = np.linspace(-3, 3, 100)
Ytest = 1 + Xtest + 0.3 * np.sin(10 * Xtest)  # Noiseless


# Define features
feature_set = {'Constant', 'Linear', 'Squared', 'Cubic', 
            'Sin(x)', 'Sin(2x)', 'Sin(5x)', 'Sin(10x)',
            'Exp(x)', 'Exp(2x)', 'Exp(5x)', 'Exp(10x)'}
#             'Log(|x|)', 'Log(2|x|)', 'Log(5|x|)', 'Log(10|x|)'}

w_hat = get_coefficient(X, Y, feature_set)
valid_error = cross_validation(X, Y, feature_set, num_folds)

num_features = len(feature_set)

plt.clf()
clear_output()
plot_opts = {'x_label': '$x$', 'y_label': '$y$', 'title': 'Num Features {}'.format(num_features), 'y_lim': [np.min(Y)-0.5, np.max(Y)+0.5]}
plot_helpers.plot_data(X, Y, options=plot_opts)

Phitest = build_features(Xtest, feature_set)
plt.plot(Xtest, Phitest @ w_hat, 'r-')

print('Feature set:', list(feature_set))
print('Coefficients', w_hat)

def remove_feature(b):
    global features, feature_set, num_features, valid_error
    worst_feature = None
    worst_feature_error = float('+Inf')
    
    # Find Worst Feature
    for feature in feature_set:
        feature_set.remove(feature)
        
        w_hat = get_coefficient(X, Y, feature_set)
        current_error = cross_validation(X, Y, feature_set, num_folds)
        
        if current_error < worst_feature_error:
            worst_feature = feature 
            worst_feature_error = current_error
        
        feature_set.add(feature) # Append feature 
    
    # If validation error increases
    if worst_feature_error > valid_error or len(feature_set) == 1:
        print('Finished! Feature set:', feature_set)
        print('Validation error:', valid_error)
        w_hat = get_coefficient(Xtrain, Ytrain, feature_set)
        print('Coefficients', w_hat)
    else:
        # Remove feature to feature set. 
        num_features -= 1 
        feature_set.remove(worst_feature)
        valid_error = worst_feature_error

        w_hat = get_coefficient(X, Y, feature_set)

        plt.clf()
        clear_output()
        display(button)
        plot_opts = {'x_label': '$x$', 'y_label': '$y$', 'title': 'Num Features {}'.format(num_features), 'y_lim': [np.min(Y)-0.5, np.max(Y)+0.5]}
        plot_helpers.plot_data(X, Y, options=plot_opts)
        
        Phitest = build_features(Xtest, feature_set)
        plt.plot(Xtest, Phitest @ w_hat, 'r-')
        print('Feature removed', worst_feature)
        print('Feature set:', list(feature_set))
        print('Coefficients', w_hat)
        

button = ipywidgets.Button(description="Remove Feature Feature")
button.on_click(remove_feature)
remove_feature(None)